<a href="https://colab.research.google.com/github/freedomtan/clip_score_on_android/blob/main/test_clip_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import requests
import tensorflow as tf

from transformers import AutoProcessor, TFCLIPVisionModel, TFCLIPTextModel, TFCLIPModel, CLIPProcessor

In [ ]:
model = TFCLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog", "a photo of two cats"], images=image, return_tensors="tf", padding="longest")
outputs = model(**inputs)

model.save('/tmp/clip_model')

loaded = tf.saved_model.load('/tmp/clip_model')
concrete_func = loaded.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape((1, 77))
concrete_func.inputs[1].set_shape((1, 77))
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

tflite_model = converter.convert()

with open('/tmp/clip_model.tflite', 'wb') as f:
    f.write(tflite_model)

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFCLIPModel.

All the weights of TFCLIPModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCLIPModel for predictions without further training.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
print(outputs.logits_per_image[0])
print(outputs.logits_per_text[0])

tf.Tensor([18.903957 11.715531 22.901352], shape=(3,), dtype=float32)
tf.Tensor([18.903957], shape=(1,), dtype=float32)


In [ ]:
import numpy as np

tflite_input_ids = inputs.input_ids.numpy()[0]
tflite_input_ids = np.append(tflite_input_ids, [tflite_input_ids[6]] * 70)
tflite_attention_mask = np.append(inputs.attention_mask.numpy()[0], [0]*70)
tflite_pixel_values = inputs.pixel_values.numpy()

i = tf.lite.Interpreter('/tmp/clip_model.tflite')
i.allocate_tensors()

# (77) -> (1, 77)
i.set_tensor(0, np.expand_dims(np.int32(tflite_attention_mask), 0))
i.set_tensor(1, np.expand_dims(tflite_input_ids, 0))
i.set_tensor(2, tflite_pixel_values)

i.invoke()

print(f'logits_per_image', i.get_tensor(i.get_output_details()[1]['index']))
print(f'logits_per_text', i.get_tensor(i.get_output_details()[2]['index']))

logits_per_image [[18.904083]]
logits_per_text [[18.904083]]


In [ ]:
np.expand_dims(np.int32(tflite_attention_mask), 0).tofile('/tmp/attention_mask.bin')
np.expand_dims(tflite_input_ids, 0).tofile('/tmp/input_ids.bin')
tflite_pixel_values.tofile('/tmp/pixel_values.bin')